### Portfolio 4: L1 vs L2 Regularization in  Linear Regression


When talking about the method to estimate the coefficents of a linear regression model the oldest and most commonly used method would be the OLS method this method evaluates the model by finding the coefficents that will minimize the sum of squared errors. In linear regression we typically have an equation of the form of $$ y = \beta_0 + \beta_1X_1 + \beta_2X_2 + \cdots + \beta_n X_n  + \epsilon $$

Using the ordinary least squares method we would find the values of $\beta$ such that the equation $\lVert \mathbf{y - X\beta} \rVert_2$ is minimized.

However the original OLS method on a model may run into some issues of overfitting when working on regression problems with lots of variables. So to reduce overfitting we can add a regularization term this term will add a penalty of  for large values of $\beta$.

Models that use l2 regularization are often called ridge regression. Where ridge regression minimizes the equation.  $\lVert \mathbf{y - X\beta} \rVert_2 + \alpha \lVert \mathbf{\beta} \rVert_2$. 

Models that use l1 regularization are often called Lasso regression. Where Lasso regression minimizes the equation.  $\lVert \mathbf{y - X\beta} \rVert_2 + \alpha \lVert \mathbf{\beta} \rVert_1$. 

Where the parameter alpha can be adjusted to give more or less regularization. A larger value of alpha results in a stronger regularization in the  L2 norm this will make the magnitude of all of the values smaller where as L1 will force more of the values to be non-zeros.

In [8]:
import pandas as pd
from sklearn.linear_model import Ridge, Lasso, LinearRegression, RidgeCV, LassoCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np
import cvxpy as cp

First we will load the dataset which we will run the linear regression on.

In [2]:
df = pd.read_csv('data/housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
#loading some more informatin on the features so we can have a better idea of what we are working with
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


It looks like there are no null values so we will not have to worry about that however I will apply some transformations to my data such as applying one-hot encoding to the categorical variable `ocean_proximity`. 

In [5]:
# Define which features are numeric and which are categorical
numeric_features = ['longitude', 'latitude','housing_median_age', 'total_rooms', 'population', 'households','median_income' ]
categorical_features = ['ocean_proximity']

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),  # Standardize numeric features
        ('cat', OneHotEncoder(), categorical_features) ,  # One-hot encode categorical features,
    ])


In [6]:
X = df.drop(columns=["median_house_value", "total_bedrooms"])
y = df['median_house_value']



In [7]:
X_transformed = preprocessor.fit_transform(X)
transformed_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
list = ['ocean_proximity_<1H OCEAN', 'ocean_proximity_INLAND',
       'ocean_proximity_ISLAND', 'ocean_proximity_NEAR BAY',
       'ocean_proximity_NEAR OCEAN']
list = numeric_features + list

X_transformed = pd.DataFrame(X_transformed, columns=list)
X_transformed

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-1.327835,1.052548,0.982143,-0.804819,-0.974429,-0.977033,2.344766,0.0,0.0,0.0,1.0,0.0
1,-1.322844,1.043185,-0.607019,2.045890,0.861439,1.669961,2.332238,0.0,0.0,0.0,1.0,0.0
2,-1.332827,1.038503,1.856182,-0.535746,-0.820777,-0.843637,1.782699,0.0,0.0,0.0,1.0,0.0
3,-1.337818,1.038503,1.856182,-0.624215,-0.766028,-0.733781,0.932968,0.0,0.0,0.0,1.0,0.0
4,-1.337818,1.038503,1.856182,-0.462404,-0.759847,-0.629157,-0.012881,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-0.758826,1.801647,-0.289187,-0.444985,-0.512592,-0.443449,-1.216128,0.0,1.0,0.0,0.0,0.0
20636,-0.818722,1.806329,-0.845393,-0.888704,-0.944405,-1.008420,-0.691593,0.0,1.0,0.0,0.0,0.0
20637,-0.823713,1.778237,-0.924851,-0.174995,-0.369537,-0.174042,-1.142593,0.0,1.0,0.0,0.0,0.0
20638,-0.873626,1.778237,-0.845393,-0.355600,-0.604429,-0.393753,-1.054583,0.0,1.0,0.0,0.0,0.0


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

Now that we have finished preprocessing the data we can finally try out the different regularization terms and compair them. First we will start with a standard OLS model that has no regularization terms. This model will simply minimize the function .... 

In [34]:
ols = LinearRegression()
ols.fit(X_train, y_train)
print(ols.score(X_test, y_test))
coefficents = ols.coef_
print(coefficents)


0.6307553884717845
[-50405.7495367  -51770.46547113  13373.01804091   -657.59789791
 -47670.77092166  53256.18909416  71680.076846   -21014.54637353
 -62752.26690878 126079.29651431 -25109.06559917 -17203.41763283]


Now we have a regularization term since we are using the ridge regregression. Ridge regression uses a l2 regularization parameter, it gives alpha* the l2 norm of the coefficents. The L2 norm minimizes the overall magnitude of the coefficents meaning the larger the value of alpha the more `regularization` will be enforced. 

Below I will test different values of alpha using a technique called cross-validation to find the optimal value of alpha for this data without using the test set. 

In [45]:
alphas = np.logspace(-5,5, 10)
cv = RidgeCV(alphas).fit(X_train , y_train)
alpha = cv.alpha_
ridge = Ridge(alpha = alpha).fit(X_train, y_train)
print(ridge.score(X_test, y_test))
print(ridge.coef_)


0.6307311751027571
[-50398.01456106 -51765.48273645  13374.50402223   -646.42540855
 -47664.51435456  53238.68304526  71675.93790434 -19353.6834757
 -61093.71620127 119427.21229916 -23439.89912238 -15539.91350237]


Now we will use the Lasso regression model. This model will try to minimize the the sum of squared error plus the alpha* the l1 norm of the coefficents. We will also use cross-validation to determine the optimal value of alpha which should be used. 

In [54]:
cv = LassoCV(cv=5).fit(X_train,y_train)
alpha = cv.alpha_
lasso = Lasso(alpha = alpha).fit(X_train, y_train)
print(lasso.score(X_test, y_test))
print(lasso.coef_)

0.6299066850703164
[-48283.88419093 -49848.11795429  13283.09612598     -0.
 -46645.0689642   51582.04018356  71499.91194567      0.
 -42519.71251891      0.          -2144.18052667   3594.7065299 ]


Looking at the accuracy results each model seems to perform the same. However if we take a look at the alpha values and coefficents we can analyze those. 

In [87]:
beta = cp.Variable(12)

obj = cp.Minimize(cp.sum_squares(X_train.values @ beta - y_train.values) + cp.norm1(beta))

prob = cp.Problem(obj)

prob.solve()

78165010823223.72

In [88]:
beta.value

array([-50405.75304392, -51770.46901988,  13373.01900107,   -657.59799137,
       -47670.77498347,  53256.19354432,  71680.0820443 , 219940.07087076,
       178202.34747044, 367033.90525427, 215845.55084095, 223751.1999044 ])

**Differences and Similarity**

I found it particulary interesting that all three of the models have a very similar accuracy score of around 63%. The main difference in the results of the three models tend to be the beta coefficents, which makes sense mathematically as we know the regularization term adds a penalty if the l1 or l2 norm is large. We can notice that when we apply l2 regualrization some of the beta values become smaller in magnitude. Where as when we apply the l1 regualrization we end up with 3 values that are 0. What is interesting is that the largest coefficent in magnitude in the other models becomes one that goes to 0. 

**Final Remarks**


I think it was very intersting to evaluate this as I have learned about linear regression in a couple different contexts such as in linear algebra class, as well as in an applied machine learning course. In the past when I had used ridge and lasso regression  I think I understood the idea that regularization was trying to prevent the model from overfitting however I did not truly understand how the techniques worked and what the difference between using a l1 regularization term vs a l2 regularization term. It seems that the l1 regularization term tries to force beta values to zero when they have small values  This is important to consider especially when dealing with a data set like this where we have the features of different scales as if we do not apply something like a standard scaler then coefficents that deal with larger values such as things like square footage would tend to be pushed towards 0 as an increase in 1 square foot would not change the price much but adding something like another room would increase the value to be much larger, for this reason it is important to first scale the data which we are working on so all the values in the dataset are within the same range. Then afterwards when we get our coefficents we can rescale back to the original format. 